## Memory

<img src="./assets/LC_Memory_before.png" width="500">

Persisting messages or 'agent state' between invocations of the agent

### Setup

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [2]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db_context: SQLDatabase

In [3]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQL command and return the results."""
    runtime = get_runtime(RuntimeContext)
    db_tool = runtime.context.db_context
    
    try:
        return db_tool.run(query)
    except Exception as e:
        return f"Error executing query: {e}"

In [4]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step by step.
- When you need data, call the tool `execute_sql` with one SELECT query.
- Read-only only: no INSERT, UPDATE, DELETE, ALTER, DROP, CREATE, REPLACE, TRUNCATE statements.
- Limit to 5 rows unless the user specifies otherwise.
- If the tool returns an error, fix the query and try again.
- Prefer explicit column lists instead of SELECT *.
"""

In [6]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model="ollama:gpt-oss:20b-cloud",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

### Run the agent

Notice the repeated queries

In [8]:
question = "This is Frank Harris. What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db_context=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris. What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (772f2c61-2d6d-41b7-9ad4-b8587282760a)
 Call ID: 772f2c61-2d6d-41b7-9ad4-b8587282760a
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (dcfd4a68-c438-40fc-a03b-9034dab82552)
 Call ID: dcfd4a68-c438-40fc-a03b-9034dab82552
  Args:
    query: SELECT CustomerId FROM Customer WHERE FirstName='Frank' AND LastName='Harris';
================================= Tool M

Now, let's ask a proceeding question

In [9]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db_context=db),
):
    step["messages"][-1].pretty_print()
    

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (85effc9b-d01f-4863-b2cb-a33a95a4da87)
 Call ID: 85effc9b-d01f-4863-b2cb-a33a95a4da87
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================

I’m not sure which “titles” you’re referring to.  
Do you mean:

- **Track titles** (from the *Track* table)?
- **Artist names** (often called titles)?
- Titles from a particular query or subset you had in mind?

Let me know which list you’d like, and I’ll pull it for you.


### Add Memory

In [10]:
from langgraph.checkpoint.memory import InMemorySaver

In [11]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model="ollama:gpt-oss:20b-cloud",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [14]:
question = "This is Frank Harris. What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    stream_mode="values",
    context=RuntimeContext(db_context=db)
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris. What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (4e1c6344-0bf1-4324-8e1f-d30938c5f52e)
 Call ID: 4e1c6344-0bf1-4324-8e1f-d30938c5f52e
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (34a715e1-a251-4b8e-baa2-b07f74782f3c)
 Call ID: 34a715e1-a251-4b8e-baa2-b07f74782f3c
  Args:
    query: SELECT CustomerId, FirstName, LastName FROM Customer WHERE FirstName='Frank' AND LastName='Harris';
===================

Now, let's ask the next question

In [15]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    stream_mode="values",
    context=RuntimeContext(db_context=db),
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (4eddf4ea-7bd6-4b3d-a9af-370099324b1a)
 Call ID: 4eddf4ea-7bd6-4b3d-a9af-370099324b1a
  Args:
    query: SELECT t.Title FROM InvoiceLine il JOIN Track t ON il.TrackId = t.TrackId WHERE il.InvoiceId=374;
================================= Tool Message =================================
Name: execute_sql

Error executing query: (sqlite3.OperationalError) no such column: t.Title
[SQL: SELECT t.Title FROM InvoiceLine il JOIN Track t ON il.TrackId = t.TrackId WHERE il.InvoiceId=374;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (41d0e30f-9418-4ba5-8c72-15b322c670a8)
 Call ID: 41d0e30f-9418-4ba5-8c72-15b322c670a8
  Args:
    query: PRAGMA table_info(Track);
========